In [17]:
import os
from gensim import corpora, models, similarities

In [18]:
import jieba
import pandas as pd
import re
import jieba.posseg as pseg   #包含詞性註記
import jieba.analyse
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

MD=pd.read_csv("C:\\Users\\Adam\\Desktop\\output\\MoneyDJ.csv",encoding="ANSI")  #讀取檔案 CSV須加編碼
MDs=MD['文章內容'] 

jieba.set_dictionary('C:\\Users\\Adam\\Desktop\\help\\dict.txt.big.txt')   #設定結巴的繁體字典
jieba.load_userdict("C:\\Users\\Adam\\Desktop\\help\\userdict.txt")  #使用者補充字典

dd=[]
for article in MDs:
    newcontent=re.sub("\d|[\s+\.\!\/_,$%^*(+\"\'《》「」]+|[+-—！，。？、~@#￥%……&*()（）:]+",'',article)
    words = jieba.cut(newcontent,cut_all=False)
    word=' '.join(words)
    dd.append(word)


dd

Building prefix dict from C:\Users\Adam\Desktop\help\dict.txt.big.txt ...
2018-06-28 16:14:38,733 : DEBUG : Building prefix dict from C:\Users\Adam\Desktop\help\dict.txt.big.txt ...
Loading model from cache C:\Users\Adam\AppData\Local\Temp\jieba.u2a5ff907f0989b23cd0cb4eed1d5c332.cache
2018-06-28 16:14:38,733 : DEBUG : Loading model from cache C:\Users\Adam\AppData\Local\Temp\jieba.u2a5ff907f0989b23cd0cb4eed1d5c332.cache
Loading model cost 1.333 seconds.
2018-06-28 16:14:40,065 : DEBUG : Loading model cost 1.333 seconds.
Prefix dict has been built succesfully.
2018-06-28 16:14:40,067 : DEBUG : Prefix dict has been built succesfully.


['新聞記者 賴宏昌 報導 英國 金融時報 日 報導 觀致 汽車 公司 透露 中國 行動 運輸 平台 商 滴滴 出行 所 改裝 的 兩台 在 上海 近郊 專用 測試 道 以及 公共 道路 上 已 進行 數個 月 之久 的 自駕車 測試 觀致 汽車 成 立於 年 為 奇瑞 與 以色列 企業 的 合資企業 中國 企業 寶能 上個月 斥資 億美元 取得 觀致 股權 並 承諾 至少 在 未來 年內 每年 向 觀致 購買 萬台車 英國 金融時報 月 日 報導 滴滴 證實 測試 中 的 自駕車 所 使用 的 軟體 是 出自 內部 團隊 硬體 則是 跟 汽車 廠商 以及 供應商 聯手 設計 滴滴 技術 長 張博 表示 自駕車 技術 將 大幅提高 運輸 效率 成為 填補 運輸 服務 供應 空檔 的 有效途徑 他 透露 滴滴 已經 低調 重壓 自駕車 有 一段時間 了 目前 在 美國 中國 的 三座 城市 進行 密集 的 測試 摩根士丹利 證券 分析師 日前 指出 任何 共乘 領導 廠商 也 必將 是 自主 駕駛 技術 的 領先者 優步 執行長 上個月 表示 優步 自主 駕駛 車 可望 在 個 月 內 上路 月 日 報導 滴滴 目前 在 美國 中國 的 三座 城市 共有 台 運行 中 的 自駕 測試 原型車 月 日 報導 滴滴 出行 於 去年底 宣布 完成 最新 一輪 超過 億美元 的 股權 融資 進而 讓 它 躍居 為 全球 最有 價值 預估 約 億美元 的 新創 企業 超越 優步 的 億美元 滴滴 透過 與 大 數據 技術 目前 已 可以 提前 分鐘 預知 需求 變化 且 準確度 高達 滴滴 技術 長 張博 表示 透過 加強 式 學習 希望 在 未來 能 到 達到 提前 小時 甚至 是 天 預知 需求 變化 的 境界 滴滴 軟銀 公司 月 日 宣布 建立 夥伴關係 雙方 將 攜手 為 計程車 產業 提供 平台 服務 這項 夥伴關係 將 利用 滴滴 的 先進 技術 去 打造 叫車 平台 藉以 提升 計程車 經營商 與 司機 的 營運 效率 滴滴 出行 於 月 日 宣布 與 下列 家 頂尖 車商 攜手 發表 開放 的 新能源 汽車 共享 平台 ： 北汽 新能源 電動車 比亞迪 長安汽車 集團 奇瑞 汽車集團 東風 乘用車 公司 中國 第一 汽車集團 吉利 汽車 華泰 汽車 江淮 汽車

In [19]:
# 建立常見字辭庫，用以移除常用字
with open("C:\\Users\\Adam\\Desktop\\help\\stop_words.txt",encoding="utf-8") as f:
    stop_word_content = f.readlines()
stop_word_content = [x.strip() for x in stop_word_content] #strip: 移除頭尾空格、中間不會
stop_word_content = " ".join(stop_word_content)   #以空格將list併在一起

In [20]:
# 建立本次文檔的語料庫(字典) 並將文檔裡的詞袋給予編號
dictionary = corpora.Dictionary(document.split() for document in dd)
stoplist = set(stop_word_content.split())  #以空格作為分隔，做出來是{}的形式
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist      #dictionary.token2id['STR'] 對照到他的token序號
            if stopword in dictionary.token2id] #有幾個id就代表有幾維向量空間
dictionary.filter_tokens(stop_ids) # 移除停用字
dictionary.compactify() #remove faps in id sequence after worfs that were removed 重編碼
#dictionary.save("lyrics/lyrics_mayday.dict")

2018-06-28 16:14:40,346 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-06-28 16:14:40,361 : INFO : built Dictionary(3017 unique tokens: ['一段時間', '一輪', '三座', '三菱', '上']...) from 40 documents (total 12726 corpus positions)


In [21]:
# 查看：序列化的結果
#for word,index in dictionary.token2id.items():   #查看字典的所有item，結果會用()包起來
#    print(word +" id:"+ str(index))

In [22]:
# 讀取字詞
texts = [[word for word in document.split() if word not in stoplist]
         for document in dd]
# 移除只出現一次的字詞 (Optional)
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [23]:
# 將corpus序列化
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize("C:\\Users\\Adam\\Desktop\\help\\news.mm", corpus) 

2018-06-28 16:14:40,785 : INFO : storing corpus in Matrix Market format to C:\Users\Adam\Desktop\help\news.mm
2018-06-28 16:14:40,788 : INFO : saving sparse matrix to C:\Users\Adam\Desktop\help\news.mm
2018-06-28 16:14:40,789 : INFO : PROGRESS: saving document #0
2018-06-28 16:14:40,801 : INFO : saved 40x2982 matrix, density=5.236% (6245/119280)
2018-06-28 16:14:40,811 : INFO : saving MmCorpus index to C:\Users\Adam\Desktop\help\news.mm.index


In [41]:
corpus

[[(2, 2),
  (3, 1),
  (4, 1),
  (5, 3),
  (6, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 5),
  (14, 1),
  (16, 1),
  (17, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 2),
  (24, 3),
  (25, 3),
  (27, 1),
  (28, 1),
  (29, 2),
  (30, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 4),
  (37, 3),
  (38, 1),
  (39, 1),
  (41, 2),
  (43, 1),
  (44, 3),
  (45, 1),
  (50, 5),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (64, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 2),
  (79, 1),
  (80, 1),
  (81, 5),
  (84, 2),
  (85, 1),
  (88, 2),
  (89, 1),
  (90, 1),
  (91, 1),
  (92, 1),
  (93, 1),
  (94, 4),
  (95, 1),
  (97, 1),
  (99, 3),
  (100, 1),
  (103, 2),
  (104, 1),
  (105, 1),
  (106, 4),
  (107, 1),
  (108, 1),
  (110, 2),
  (111, 1),
  (112, 2),
  (113, 1),
  (114, 2),
  (115, 1),
  (118, 1),
  (119, 1),
  

In [24]:
# 創建 tfidf model
tfidf = models.TfidfModel(corpus)
# 轉為向量表示
corpus_tfidf = tfidf[corpus]

2018-06-28 16:14:40,951 : INFO : collecting document frequencies
2018-06-28 16:14:40,952 : INFO : PROGRESS: processing document #0
2018-06-28 16:14:40,954 : INFO : calculating IDF weights for 40 documents and 2981 features (6245 matrix non-zeros)


In [39]:
#建立lsi模型的時候：需要給定tfidf生成的語料庫(corpus)、給定字典(dictionary)、和制定主題數

# 創建 LSI model 潛在語義索引
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)
corpus_lsi = lsi[corpus_tfidf] # LSI潛在語義索引
lsi.save("C:\\Users\\Adam\\Desktop\\help\\news.lsi")
corpora.MmCorpus.serialize("C:\\Users\\Adam\\Desktop\\help\\news.mm", corpus_lsi)
print("LSI topics:")
lsi.print_topics(10)

2018-06-28 16:36:13,848 : INFO : using serial LSI version on this node
2018-06-28 16:36:13,850 : INFO : updating model with new documents
2018-06-28 16:36:13,889 : INFO : preparing a new chunk of documents
2018-06-28 16:36:13,892 : INFO : using 100 extra samples and 2 power iterations
2018-06-28 16:36:13,893 : INFO : 1st phase: constructing (2985, 110) action matrix
2018-06-28 16:36:13,896 : INFO : orthonormalizing (2985, 110) action matrix
2018-06-28 16:36:13,958 : INFO : 2nd phase: running dense svd on (110, 40) matrix
2018-06-28 16:36:13,963 : INFO : computing the final decomposition
2018-06-28 16:36:13,964 : INFO : keeping 10 factors (discarding 59.668% of energy spectrum)
2018-06-28 16:36:13,967 : INFO : processed documents up to #40
2018-06-28 16:36:13,969 : INFO : topic #0(1.676): -0.167*"利率" + -0.145*"第季" + -0.122*"美股" + -0.121*"指數" + -0.112*"預估" + -0.105*"公債" + -0.102*"殖" + -0.101*"滴滴" + -0.101*"波動" + -0.100*"年"
2018-06-28 16:36:13,971 : INFO : topic #1(1.395): 0.240*"委託人" + 0

LSI topics:


[(0,
  '-0.167*"利率" + -0.145*"第季" + -0.122*"美股" + -0.121*"指數" + -0.112*"預估" + -0.105*"公債" + -0.102*"殖" + -0.101*"滴滴" + -0.101*"波動" + -0.100*"年"'),
 (1,
  '0.240*"委託人" + 0.239*"公司" + 0.239*"本" + 0.192*"通知" + 0.192*"違約" + 0.180*"發生" + 0.178*"兆豐" + 0.176*"公告" + 0.159*"措施" + 0.137*"股份"'),
 (2,
  '-0.477*"滴滴" + -0.156*"計程車" + 0.147*"利率" + -0.130*"汽車" + -0.128*"服務" + -0.120*"軟銀" + -0.114*"平台" + -0.111*"出行" + 0.110*"美股" + -0.106*"夏普"'),
 (3,
  '0.405*"滴滴" + -0.193*"早盤" + -0.169*"一部" + -0.165*"日圓" + -0.149*"收盤" + -0.144*"第季" + -0.143*"家數" + 0.133*"計程車" + -0.118*"東證" + -0.110*"夏普"'),
 (4,
  '0.257*"第季" + -0.213*"早盤" + 0.185*"預估" + -0.175*"一部" + 0.161*"萬噸" + -0.160*"日圓" + -0.150*"家數" + -0.124*"東證" + -0.108*"夏普" + 0.107*"值"'),
 (5,
  '-0.375*"黃金" + -0.278*"公噸" + 0.249*"第季" + -0.221*"資產" + -0.175*"金價" + -0.139*"增加" + -0.134*"萬噸" + -0.132*"基金" + 0.128*"利率" + -0.120*"流入"'),
 (6,
  '0.311*"蘋果" + -0.227*"滴滴" + 0.157*"產品" + 0.148*"夏普" + 0.148*"開賣" + 0.144*"廉價版" + 0.141*"顯示器" + -0.127*"第季" + 0.121*"研發" 

In [40]:
# 基於tfidf-> lsi 的文本相似度分析
doc = "網站 創辦人 月 日 曾 直指 標準 普爾 指數 出現 敗象 如今 標普 已 從 月 日 的 歷史 高 短短 日 就 摔 逾 進入 修正 區認 為 美股 以 市場 恐慌 情緒 測試 出 低點 後 接下來 有望 恢復 上升 走勢 曾 於 月 日 在 指出 標普月 日 跳 漲 逾 拉 升至 歷史 高 但 其中 僅 不到 的 成分股 走強 是 相當 詭異 的 現象 在 年 這樣 的 現象 出現 過 一次 年則 出現 過 三次 而 在 這 四次 中標 普都 立即 拉 回 最終 大幅 修正 進入 熊市 日 在 部落 格 發文 表示 見 此標 普本波 修正 來得 極快 且 跌幅 一下子 就 超越 至今 尚未 止跌 那麼 現在 究竟 該 買 還是 該 賣 從 標普 期貨 走勢 來 分析 發現 指標 已經 嚴重 超賣 股市 極 可能 出現 反彈 根據 他 月 日 發布 的 報告 標普 期貨 已 觸及 日 移動 平均線 還 來到 費波 南希 係數 的 折返 點且 從 高點 至 低點 的 跌幅 也 高達 遠 多於 他 原先 預估 的 修正 幅 比較 接近 過去 四個 循環 的 平均 跌幅 來到 中期 低點 相 較之 下標 普 現貨 的 拉 回 幅度 還 只有 很 顯然 股市 本波 走勢 並無 道理 可循 不過 幾乎 所有 觀察 的 標的 都 暗示 美股 修正 結束 後 會 再次 走多 標普 期貨 已經 滿足 他 預設 的 下方 目標 標普 現貨 則還 沒 理論 上美股 應會 透過 恐慌 拋售 潮 測試 低點 如此一來 買進 訊號 才 有 機會 現身 也就是說 他 認為 投資人 仍 可 繼續執行 拉回 就 買進 的 策略 過去 多空 建議 的 紀錄 見此 投資 策略 長 月底 曾 在 股災 引爆 前 警告 今年 美股 可能 恐怖 拉 回 如今 他 又 出面 直指 在 更 多 投資人 恐慌 殺出 或 全面 投降 前 股市 恐難 觸底 日 報導 見此 在 接受 專訪 時 依舊 預測 美股 將拉回 他 聲稱 上週五 月 日 股災 首度 引爆 道瓊 工業 平均 指數 慘跌 近點 之際 投資人 並未 瘋狂 湧進 公債 美元 或 黃金 避險 防衛 型 股票 也 並未 跳高 估計 美股 還得 再 跌 一段 才能 引發 股市 觸底 所 需 的 恐懼 氣氛 和 投降 拋股潮 認為 投資人 開始 重新 調整 市場 估值 估值 在 下 修 過後 股 債市 才能 繼續 延續 多頭 走勢 月 日 在 接受 專訪 時 曾 說 企業 消費者 和 投資人 爆發 了 樂觀 情緒 人們 把 市場 估值 推得 太高 完全 忽略 利空 年 美股 可能 會 有 非常 恐怖 的 拉 回 走勢 預測 跌幅 將 接近 他 直指 年期 公債 殖 利率 上揚 股票 本益比 偏高 經濟 和 企業 盈餘 無法 持續 帶來 驚喜 將 導致 股市 重挫 然而 他 不 相信 這是 經濟 將 陷入 衰退 的 徵兆 預期 全球 同步 復甦 美國 總統 川普 推出 對 企業 友善 的 政策 將讓 美國 未來 幾年 不 受 衰退 威脅 因此 股市 若 有 拉 回會 是 極佳 的 逢 低進 場點 ＊ 編者按 ： 本文 僅供參考 之用 並不 構成 要約 招攬 或 邀請 誘使 任何 不論 種類 或 形式 之 申述 或 訂立 任何 建議 及 推薦 讀者 務請 運用 個人 獨立思考 能力 自行 作出 投資 決定 如 因 相關 建議 招致 損失 概與 精實 財經 媒體 編者 及 作者 無涉 "
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lsi = lsi[vec_bow] # 用前面建好的 lsi 模型去計算這一篇歌詞 (input: 斷詞後的詞包、output: 20個主題成分)
print(vec_lsi)

[(0, -15.798722468246435), (1, -4.179485051776739), (2, 6.6834727266078939), (3, 5.6468903630769161), (4, -5.3923094794090867), (5, 1.0809117409396487), (6, 1.5850939889937095), (7, 0.90312162170404375), (8, 10.098992049724119), (9, -0.084454221036736624)]


In [27]:
# 建立索引
index = similarities.MatrixSimilarity(lsi[corpus]) 


# 計算相似度（前五名）
sims = index[vec_lsi] 
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:5])

2018-06-28 16:14:41,465 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-06-28 16:14:41,469 : INFO : creating matrix with 40 documents and 3 features


[(31, 0.99999982), (35, 0.99831319), (9, 0.99778676), (4, 0.99569696), (29, 0.99567735)]


In [28]:
lyrics = [];

for i, line in enumerate(dd):
    lyrics.append(line)

for lyric in sims[:5]:
    print("\n相似新聞：",  lyrics[lyric[0]])
    print("相似度：",  lyric[1])


相似新聞： 新聞記者 郭妍希 報導 曾 獲霸榮 網站 評為 紀錄 優良 交易員 的 網站 創辦人 月 日 曾 直指 標準 普爾 指數 出現 敗象 如今 標普 已 從 月 日 的 歷史 高 短短 日 就 摔 逾 進入 修正 區認 為 美股 以 市場 恐慌 情緒 測試 出 低點 後 接下來 有望 恢復 上升 走勢 曾 於 月 日 在 指出 標普月 日 跳 漲 逾 拉 升至 歷史 高 但 其中 僅 不到 的 成分股 走強 是 相當 詭異 的 現象 在 年 這樣 的 現象 出現 過 一次 年則 出現 過 三次 而 在 這 四次 中標 普都 立即 拉 回 最終 大幅 修正 進入 熊市 日 在 部落 格 發文 表示 見 此標 普本波 修正 來得 極快 且 跌幅 一下子 就 超越 至今 尚未 止跌 那麼 現在 究竟 該 買 還是 該 賣 從 標普 期貨 走勢 來 分析 發現 指標 已經 嚴重 超賣 股市 極 可能 出現 反彈 根據 他 月 日 發布 的 報告 標普 期貨 已 觸及 日 移動 平均線 還 來到 費波 南希 係數 的 折返 點且 從 高點 至 低點 的 跌幅 也 高達 遠 多於 他 原先 預估 的 修正 幅 比較 接近 過去 四個 循環 的 平均 跌幅 來到 中期 低點 相 較之 下標 普 現貨 的 拉 回 幅度 還 只有 很 顯然 股市 本波 走勢 並無 道理 可循 不過 幾乎 所有 觀察 的 標的 都 暗示 美股 修正 結束 後 會 再次 走多 標普 期貨 已經 滿足 他 預設 的 下方 目標 標普 現貨 則還 沒 理論 上美股 應會 透過 恐慌 拋售 潮 測試 低點 如此一來 買進 訊號 才 有 機會 現身 也就是說 他 認為 投資人 仍 可 繼續執行 拉回 就 買進 的 策略 過去 多空 建議 的 紀錄 見此 投資 策略 長 月底 曾 在 股災 引爆 前 警告 今年 美股 可能 恐怖 拉 回 如今 他 又 出面 直指 在 更 多 投資人 恐慌 殺出 或 全面 投降 前 股市 恐難 觸底 日 報導 見此 在 接受 專訪 時 依舊 預測 美股 將拉回 他 聲稱 上週五 月 日 股災 首度 引爆 道瓊 工業 平均 指數 慘跌 近點 之際 投資人 並未 瘋狂 湧進 公債 美元 或 黃金 避險 防衛 型 股票 也 並未 跳高 估計 美股 還得 再 跌 一段 才能 引發

In [29]:
# 創建 LDA model
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=3)
corpus_lda = lda[corpus_tfidf] # LDA潛在語義索引

print("LDA topics:")
lda.print_topics(4)

2018-06-28 16:14:41,702 : INFO : using symmetric alpha at 0.3333333333333333
2018-06-28 16:14:41,703 : INFO : using symmetric eta at 0.3333333333333333
2018-06-28 16:14:41,705 : INFO : using serial LDA version on this node
2018-06-28 16:14:41,708 : INFO : running online (single-pass) LDA training, 3 topics, 1 passes over the supplied corpus of 40 documents, updating model once every 40 documents, evaluating perplexity every 40 documents, iterating 50x with a convergence threshold of 0.001000
2018-06-28 16:14:41,710 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-06-28 16:14:41,830 : INFO : -20.740 per-word bound, 1751367.2 perplexity estimate based on a held-out corpus of 40 documents with 319 words
2018-06-28 16:14:41,831 : INFO : PROGRESS: pass 0, at document #40/40
2018-06-28 16:14:41,882 : INFO : topic #0 (0.333): 0.001*"公噸" + 0.001*"夏普" + 0.001*"黃金" + 0.001*"日圓" + 0.001*"美股" + 0.001*"一部" + 0.

LDA topics:


[(0,
  '0.001*"公噸" + 0.001*"夏普" + 0.001*"黃金" + 0.001*"日圓" + 0.001*"美股" + 0.001*"一部" + 0.001*"收盤" + 0.001*"解除" + 0.001*"合作" + 0.001*"高通"'),
 (1,
  '0.001*"電視" + 0.001*"第季" + 0.001*"率" + 0.001*"預估" + 0.001*"波動" + 0.001*"資產" + 0.001*"貨幣" + 0.001*"億美元" + 0.001*"年" + 0.001*"指數"'),
 (2,
  '0.001*"滴滴" + 0.001*"利率" + 0.001*"萬噸" + 0.001*"蘋果" + 0.001*"大陸" + 0.001*"祖馬" + 0.001*"公債" + 0.001*"黃金" + 0.001*"公司" + 0.001*"機器人"')]

In [30]:

vec_lda = lda[vec_bow] # 用前面建好的 lda 去計算這一篇歌詞

sims_2 = index[vec_lda] # 將已經算完tfidf的字詞轉為lsi #基於lsi的文本相似度分析
sims_2 = sorted(enumerate(sims_2), key=lambda item: -item[1])
print(sims_2[:5])

[(4, 0.29273427), (9, 0.28389934), (29, 0.2790609), (38, 0.27796206), (35, 0.25412354)]


In [31]:
lyrics_2 = [];

for i, line in enumerate(dd):
    lyrics_2.append(line)

for lyric_2 in sims_2[:5]:
    print("\n相似新聞：",  lyrics_2[lyric_2[0]])
    print("相似度：",  lyric_2[1])


相似新聞： 新聞記者 陳 苓 報導 上週四 日為止 道 瓊 工業 指數 一度 下 殺點 自 高點 重挫 陷入 修正 領域 儘管 上週五 日美股 止跌回升 投資人 依舊 焦慮不安 分析師 認為 可能 是殖 利率 攀升 讓 債市 重獲 青睞 正 和 股市 爭寵 不過 美股 基本面 仍強 最多 回檔 巴倫 報導 有 一派 人士 認為 股市 大跌 的 癥結 是 利率 上揚 債券 變得 更具 吸引力 搶走 股市 風采 首席 市場 策略師 說 多 年來 人們 都 認為 股票 比 債券 便宜 如今 殖 利率 上升 由於 殖 利率 和 債券 價格 呈 反向 走勢 殖 利率 走升 代表 債券 價格 下跌 變得 更具 競爭力 股市 比 債市 便宜 的 指標 將 重新 計價 多名 分析師 指出 市場 仍然 強健 無須 過於 恐慌 波動性 飆高 不是 基本面 問題 造成 投資 長 表示 美股 應該 不會 大跌 最糟 情況 下 可能 回檔 之前 美股 漲勢 又 兇 又 猛 出現 回檔 相當 正常 也 有利 股市 資深 投資 編輯 指出 美股 跌幅 多寡 要 看 股民 聚焦 恐懼 或 基本面 恐懼 會 讓 股市 續跌 不過 屆時 聰明 錢會 進場 撈底 他 估計 如果 美股 跌破 日 移動 平均線 大約 點算 是 超跌 上次 美股 跌破 日 均線 是 在 年 年 中 當時 是 金融 系統 崩盤 引發 的 熊市 和 目前 情況 不同 多數 專家 認為 股市 會 在 近期內 觸底 逐漸 適應 利率 飆高 的 事實 不過 打底 過程 將 劇烈 震盪 美銀 美林 首席 投資 策略師 估計 標普 指數 跌至點 美國 年期 公債 殖 利率 升至 時 買盤 將會 進場 新聞報導 道瓊 工業 平均 指數 日 漲點 或 收點 ； 週線 下跌 標準 普爾 指數 日 漲點 或 收點 ； 週線 下跌 ＊ 編者按 ： 本文 僅供參考 之用 並不 構成 要約 招攬 或 邀請 誘使 任何 不論 種類 或 形式 之 申述 或 訂立 任何 建議 及 推薦 讀者 務請 運用 個人 獨立思考 能力 自行 作出 投資 決定 如 因 相關 建議 招致 損失 概與 精實 財經 媒體 編者 及 作者 無涉
相似度： 0.292734

相似新聞： 新聞記者 陳 苓 報導 美股 持續 震盪 投資人 湧向 低 風險 的 債市 避難 此外 市